In [6]:
import pandas as pd

import numpy as np
import scipy as sp

from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [2]:
import gensim
import sklearn

In [3]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

GEMINI_API_KEY =  os.getenv('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

d:\1UGM\SEM 5\COURSES\STBI\Tugas Akhir\healthcare-chatbot-using-qa-with-word-embedding-ir\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df_medquad = pd.read_csv("MedQuad-MedicalQnADataset.csv")
df_medquad.head()

,Unnamed: 0,qtype,Question,Answer
0,0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [40]:
df_pubmed = pd.read_csv("PubMed_200k_RCT/train.csv")
df_pubmed.head()

,abstract_id,line_id,abstract_text,line_number,total_lines,target
0,24491034,24491034_0_11,The emergence of HIV as a chronic condition me...,0,11,BACKGROUND
1,24491034,24491034_1_11,This paper describes the design and evaluation...,1,11,BACKGROUND
2,24491034,24491034_2_11,This study is designed as a randomised control...,2,11,METHODS
3,24491034,24491034_3_11,The intervention group will participate in the...,3,11,METHODS
4,24491034,24491034_4_11,The program is based on self-efficacy theory a...,4,11,METHODS


# Preprocessing

In [43]:
df_pubmed = df_pubmed.dropna(subset=['abstract_text'])  # Remove rows with NaN in abstract_text
df_pubmed = df_pubmed.sort_values(by=['line_id'])
df_pubmed = df_pubmed.groupby('abstract_id').agg({'abstract_text':' '.join}).reset_index()
df_pubmed.head()

,abstract_id,abstract_text
0,1279170,We conducted this study to assess the clinical...
1,1281030,To determine whether prophylactic treatment wi...
2,1282364,After the discovery of type C hepatitis virus ...
3,1283117,Since it is not clear whether testosterone or ...
4,1283730,The aim was to study the pharmacokinetic param...


In [44]:
# add qa id to medquad
df_medquad['qa_id'] = range(1, len(df_medquad) + 1)
df_medquad = df_medquad[['qa_id', 'Question', 'Answer', 'qtype']]
df_medquad.head()

,qa_id,Question,Answer,qtype
0,1,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...,susceptibility
1,2,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...,symptoms
2,3,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...,susceptibility
3,4,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos...",exams and tests
4,5,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen...",treatment


In [45]:
train_medquad, test_medquad = train_test_split(df_medquad, test_size=0.2, random_state=42)

# Word Embedding

In [46]:
model_path = 'GoogleNews-vectors-negative300.bin'
word2vec = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [47]:
def preprocess(sentence):
    return [word for word in sentence.lower().split() if word in word2vec]

def get_sentence_vector(sentence):
    words = preprocess(sentence)
    if words:
        return np.mean([word2vec[word] for word in words], axis=0)
    else:
        return np.zeros(word2vec.vector_size)

In [48]:
df_pubmed['abstract_vector'] = df_pubmed['abstract_text'].apply(get_sentence_vector)
train_medquad['question_vector'] = train_medquad['Question'].apply(get_sentence_vector)

In [49]:
df_pubmed

,abstract_id,abstract_text,abstract_vector
0,1279170,We conducted this study to assess the clinical...,"[-0.024794796, 0.067829445, -0.0046317396, 0.0..."
1,1281030,To determine whether prophylactic treatment wi...,"[-0.084130615, 0.08988233, 0.01364074, 0.04562..."
2,1282364,After the discovery of type C hepatitis virus ...,"[0.0002311631, 0.074568965, 0.009816241, 0.071..."
3,1283117,Since it is not clear whether testosterone or ...,"[-0.022251092, 0.055196468, 0.02946, 0.0389862..."
4,1283730,The aim was to study the pharmacokinetic param...,"[-0.04548918, 0.06224963, 0.02575707, 0.064899..."
...,...,...,...
190649,26521577,To compare the differences in the clinical eff...,"[-0.0071094367, 0.07761174, 0.022264183, 0.046..."
190650,26521581,To observe the efficacy of acupuncture on pain...,"[-0.019650908, 0.06335896, 0.009455694, 0.0609..."
190651,26521582,To explore the clinical effect of Erlong Xizhu...,"[0.009212655, 0.03145588, 0.010136922, 0.06303..."
190652,26521589,To observe the myocardial protective effect of...,"[-0.023100011, 0.06358117, 0.022341112, 0.0516..."


In [50]:
train_medquad

,qa_id,Question,Answer,qtype,question_vector
2366,2367,What is (are) Treatment Methods for Kidney Fai...,Peritoneal dialysis is a treatment for kidney ...,information,"[-0.10026169, 0.14516449, 0.11820221, 0.018508..."
12203,12204,Is Renal oncocytoma inherited ?,Is a renal oncocytoma inherited? Most renal o...,inheritance,"[-0.0730896, 0.23470052, 0.056274414, 0.037801..."
8054,8055,How many people are affected by 3-M syndrome ?,3-M syndrome is a rare disorder. About 50 indi...,frequency,"[0.071114674, 0.011885507, 0.009992327, 0.1079..."
9934,9935,How many people are affected by Fanconi anemia ?,"Fanconi anemia occurs in 1 in 160,000 individu...",frequency,"[0.053083148, 0.02039555, 0.0564488, 0.0827985..."
6405,6406,What are the genetic changes related to Alport...,"Mutations in the COL4A3, COL4A4, and COL4A5 ge...",genetic changes,"[-0.022216797, 0.06273542, -0.02192906, 0.0864..."
...,...,...,...,...,...
11284,11285,What are the symptoms of Lipoic acid synthetas...,What are the signs and symptoms of Lipoic acid...,symptoms,"[-0.02408273, 0.091526575, 0.098336354, 0.0777..."
11964,11965,What is (are) Cerebellar degeneration ?,Cerebellar degeneration refers to the deterior...,information,"[0.031791687, 0.07060242, 0.013916016, 0.08792..."
5390,5391,What is (are) Rashes ?,A rash is an area of irritated or swollen skin...,information,"[-0.0126241045, 0.0038045247, 0.11140951, 0.07..."
860,861,What is the outlook for Syringomyelia ?,"Symptoms usually begin in young adulthood, wit...",outlook,"[0.007797241, -0.0023701985, 0.045979816, 0.05..."


# Information Retrieval

In [51]:
def find_top_k_abstracts(query_vector, abstract_vectors_col, k=3, df=df_pubmed):
    cosine_scores = cosine_similarity([query_vector], np.array(df[abstract_vectors_col]).tolist())[0]
    top_k_indices = np.argsort(cosine_scores)[-k:][::-1]
    return df.iloc[top_k_indices]['abstract_id'].values, cosine_scores[top_k_indices]

def find_top_k_answers(df, query_vector, question_vectors_col, k=3):
    cosine_scores = cosine_similarity([query_vector], np.array(df[question_vectors_col]).tolist())[0]
    top_k_indices = np.argsort(cosine_scores)[-k:][::-1]
    return df.iloc[top_k_indices]['qa_id'].values, cosine_scores[top_k_indices]

def get_qa_pairs(df, qa_ids):
    return df[df['qa_id'].isin(qa_ids)][['qa_id', 'Question', 'Answer']]


def get_abstracts(abstract_ids):
    return df_pubmed[df_pubmed['abstract_id'].isin(abstract_ids)][['abstract_id', 'abstract_text']]

def query_top_k_answers_and_abstracts(df, query, k=10):
    query_vector = get_sentence_vector(query)

    qa_ids, question_scores = find_top_k_answers(df, query_vector, 'question_vector', k)
    abstract_ids, abstract_scores = find_top_k_abstracts(query_vector, 'abstract_vector', k)

    answers_df = get_qa_pairs(df, qa_ids)
    abstracts_df = get_abstracts(abstract_ids)

    answers_df['Similarity Score'] = question_scores
    abstracts_df['Similarity Score'] = abstract_scores

    return answers_df, abstracts_df

In [52]:
queries = [
    "What is the incubation period of COVID-19?",
    "Cure for fever?",
    "Who is at risk for Lymphocytic Choriomeningit",
    "What are the symptoms of Ligma?"
]

In [53]:
for query in queries:
    print(f"Query: {query}")
    answers_df, abstracts_df = query_top_k_answers_and_abstracts(train_medquad, query)
    print("Top Answers:")
    display(answers_df)
    print("Top Abstracts:")
    display(abstracts_df)
    print("\n")

Query: What is the incubation period of COVID-19?
Top Answers:


,qa_id,Question,Answer,Similarity Score
10753,10754,What is (are) dyskeratosis congenita ?,Dyskeratosis congenita is a disorder that can ...,0.647081
252,253,what is the treatment for vancomycin-resistant...,On this Page General Information What is vanco...,0.637043
11895,11896,What is (are) Anonychia congenita ?,Anonychia congenita is an extremely rare nail ...,0.637043
15727,15728,What is (are) Pachyonychia congenita ?,Pachyonychia congenita (PC) is a rare inherite...,0.637043
8918,8919,What is (are) pachyonychia congenita ?,Pachyonychia congenita is a condition that pri...,0.637043
7678,7679,What is (are) paramyotonia congenita ?,Paramyotonia congenita is a disorder that affe...,0.637043
10058,10059,What is (are) Waldenstrm macroglobulinemia ?,Waldenstrm macroglobulinemia is a rare blood c...,0.637043
392,393,What is the outlook for Agenesis of the Corpus...,Prognosis depends on the extent and severity o...,0.633984
12772,12773,What is (are) Paramyotonia congenita ?,Paramyotonia congenita is an inherited conditi...,0.631612
728,729,What is the outlook for Neurosyphilis ?,Prognosis can change based on the type of neur...,0.630091


Top Abstracts:


,abstract_id,abstract_text,Similarity Score
8088,8612858,To establish whether time to down-regulation a...,0.685417
26428,10748773,Irrigation suction drainage ( ISD ) is an addi...,0.684163
44142,12526238,Heart rate has been used to measure infants ' ...,0.683375
50652,14601817,The purpose of this study was to investigate w...,0.682391
59581,15561795,To measure the impact of a computerized guidel...,0.676083
93172,18295761,To determine the optimum time interval between...,0.674333
96992,18577202,Two types of methods are used to assess learni...,0.672660
122779,20696729,The goal was to assess the feasibility of earl...,0.672455
139950,22068638,Near infrared ( NIR ) spectroscopy is a techno...,0.672451
186231,25432920,Does culture in a closed system result in an i...,0.671429




Query: Cure for fever?
Top Answers:


...

,abstract_id,abstract_text,Similarity Score
28829,10975790,1-2 % of all patients under non-steroidal anti...,0.811170
33487,11403365,The Hypertension Optimal Treatment ( HOT ) Stu...,0.794185
61870,15742336,The efficacy of allogeneic hematopoietic stem ...,0.790489
116096,20139767,This study evaluates the Alzheimer disease ris...,0.784338
117211,20216073,To determine whether family medical history as...,0.783373
152881,22992357,"Family Healthware , a tool developed by the CD...",0.782582
153661,23046591,Atrial fibrillation ( AF ) is the most common ...,0.781664
177495,24686885,There is little evidence to inform the targete...,0.781331
183666,25188543,Civilian posttraumatic stress disorder ( PTSD ...,0.780802
186701,25467619,To determine the perceived risk of type 2 diab...,0.780116




Query: What are the symptoms of Ligma?
Top Answers:


,qa_id,Question,Answer,Similarity Score
14104,14105,What are the symptoms of Phacomatosis pigmento...,What are the signs and symptoms of phacomatosi...,1.0
13641,13642,What are the symptoms of Camptodactyly taurinu...,What are the signs and symptoms of Camptodacty...,1.0
15548,15549,What are the symptoms of Coccygodynia ?,What signs and symptoms are associated with co...,1.0
14526,14527,What are the symptoms of GM1 gangliosidosis ?,What are the signs and symptoms of GM1 ganglio...,1.0
13644,13645,What are the symptoms of Tetramelic monodactyly ?,What are the signs and symptoms of Tetramelic ...,1.0
15074,15075,What are the symptoms of Syringoma ?,What are the signs and symptoms of Syringoma? ...,1.0
1787,1788,What are the symptoms of Cystocele ?,The symptoms of a cystocele may include\n ...,1.0
15997,15998,What are the symptoms of Microhydranencephaly ?,What are the signs and symptoms of Microhydran...,1.0
16349,16350,What are the symptoms of Microtia-Anotia ?,What are the signs and symptoms of Microtia-An...,1.0
11302,11303,What are the symptoms of D-glycericacidemia ?,What are the signs and symptoms of D-glycerica...,1.0


Top Abstracts:


,abstract_id,abstract_text,Similarity Score
13777,9300508,The majority of patients presenting to cardiac...,0.770450
32707,11329526,Although the patient experiences the symptoms ...,0.759774
59762,15572868,"According to homeopathic theory , symptoms pro...",0.756948
64519,15938885,To investigate the states of chronic symptoms ...,0.756407
81392,17291177,Expectancy and modeling have been cited as fac...,0.755323
87981,17805217,Generalized anxiety disorder ( GAD ) is a chro...,0.753190
99741,18806203,"Occasional leg symptoms , like feelings of hea...",0.751274
102943,19127072,Subjects with atopic syndrome often perceive s...,0.749022
117119,20211300,The timely and accurate identification of symp...,0.748671
144191,22364776,Tobacco withdrawal symptoms may be confounded ...,0.747349


# Pass Context to LLM

In [54]:
# def call_llm(context, query):
#     prompt = f"Answer the following medical question based on the provided context."

#     prompt += f"\nQuestion: {query}\n"

#     prompt += "\nRelevant Medical QA Pairs::\n"
#     for i, qa in enumerate(context['answers'], 1):
#         prompt += f"\nQA Pair {i}:\nQ: {qa['Question']}\nA: {qa['Answer']}\n"
    
#     prompt += "\nRelevant Medical Research Abstracts:\n"
#     for i, abstract in enumerate(context['abstracts'], 1):
#         prompt += f"\nAbstract {i}:\n{abstract['abstract_text']}\n"
    
#     prompt += "\nBased on the above context, do not make up an answer and do not use your existing knowledge, please provide a comprehensive answer to the question."

#     response = model.generate_content(prompt)
    
#     return response 

In [55]:
# Changed Prompt
def call_llm(context, query):
    prompt = f"You are a medical expert assistant. Answer the following medical question comprehensively and accurately. If the provided context contains relevant information, use it. If not, use your general medical knowledge to provide the best possible answer."

    prompt += f"\nQuestion: {query}\n"

    prompt += "\nContext:\n"
    prompt += "\nRelevant Medical QA Pairs:\n"
    for i, qa in enumerate(context['answers'], 1):
        prompt += f"\nQA Pair {i}:\nQ: {qa['Question']}\nA: {qa['Answer']}\n"
    
    prompt += "\nRelevant Medical Research Abstracts:\n"
    for i, abstract in enumerate(context['abstracts'], 1):
        prompt += f"\nAbstract {i}:\n{abstract['abstract_text']}\n"
    
    prompt += "\nProvide a clear, direct, and comprehensive answer to the question. Focus on being helpful and informative to the user."

    response = model.generate_content(prompt)
    
    return response 

In [56]:
def process_medical_query(df, query):
    answers_df, abstracts_df = query_top_k_answers_and_abstracts(df, query)
    
    if len(answers_df) == 0 and len(abstracts_df) == 0:
        return {
            'response': "I apologize, but I don't have enough reliable information to answer this question.",
            'confidence': 'low'
        }
    
    context = {
        'answers': answers_df.to_dict(orient='records'),
        'abstracts': abstracts_df.to_dict(orient='records')
    }
    
    llm_response = call_llm(context, query)
    
    return {
        'response': llm_response,
        'confidence': 'high'
    }

test_query = "Who is at risk for Lymphocytic Choriomeningit?"
result = process_medical_query(train_medquad, test_query)
print(result['response'])

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Individuals of all ages who come into contact with the urine, feces, saliva, or blood of wild mice are at risk of Lymphocytic Choriomeningitis (LCM) infection.  Pet mice and hamsters from contaminated colonies or those infected by wild mice can also transmit the virus to their owners.  Furthermore, pregnant women can transmit the infection to their fetuses (vertical transmission).  Laboratory workers handling the virus or infected animals are also at increased risk, though this can be mitigated with proper safety precautions and the use of virus-tested animals.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.10437754038217906
        }
      ],
      "usage_metadata": 

# Evaluation

In [57]:
import time
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(
    stop=stop_after_attempt(3),  # Retry 3 times
    wait=wait_exponential(multiplier=1, min=4, max=10)  # Wait between retries
)
def process_medical_query_with_retry(df, query):
    try:
        return process_medical_query(df, query)
    except Exception as e:
        if "429" in str(e):  # Rate limit error
            time.sleep(2)  # Wait 2 seconds before retry
            raise  # Retry through decorator
        raise  # Other errors

In [58]:
from sklearn.metrics import f1_score
import numpy as np
from rouge_score import rouge_scorer


def preprocess_text(text):
    """
    Simple tokenization function that splits on spaces and removes punctuation
    """
    if not isinstance(text, str):
        return set()
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove basic punctuation
    for punct in '.,!?;:()[]{}""\'':
        text = text.replace(punct, ' ')
    
    # Split on whitespace and filter out empty tokens
    tokens = [token.strip() for token in text.split()]
    tokens = [token for token in tokens if token]
    
    return set(tokens)

def calculate_f1(predicted_answer, true_answer):
    """
    Calculate F1 score by checking if all words from true answer appear in predicted answer
    """
    if not isinstance(predicted_answer, str) or not isinstance(true_answer, str):
        return 0
    
    # Convert to lowercase
    pred_text = predicted_answer.lower()
    true_text = true_answer.lower()
    
    # Get unique words from true answer
    true_words = set(true_text.split())
    
    # Count how many true answer words appear in predicted answer
    matches = sum(1 for word in true_words if word in pred_text)
    
    # Calculate precision and recall
    if len(true_words) == 0:
        return 0
    
    recall = matches / len(true_words)
    precision = matches / len(true_words)  # Using same denominator as we only care about true answer words
    
    # Calculate F1
    if precision + recall == 0:
        return 0
    f1 = 2 * (precision * recall) / (precision + recall)
    
    return f1

def evaluate_response(predicted_answer, true_answer):
    """
    Evaluate response using F1 and ROUGE metrics
    """
    results = {}
    
    # Calculate F1 score
    f1 = calculate_f1(predicted_answer, true_answer)
    results['f1_score'] = f1
    
    # Calculate ROUGE scores (modified to be more lenient)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, predicted_answer)
    
    # Adjust ROUGE scores based on word presence
    true_words = set(true_answer.lower().split())
    pred_lower = predicted_answer.lower()
    word_match_ratio = sum(1 for word in true_words if word in pred_lower) / len(true_words) if true_words else 0
    
    results['rouge1'] = max(rouge_scores['rouge1'].fmeasure, word_match_ratio)
    results['rouge2'] = rouge_scores['rouge2'].fmeasure
    results['rougeL'] = max(rouge_scores['rougeL'].fmeasure, word_match_ratio)
    
    return results

def evaluate_model(num_samples=10, random_seed=42):
    """
    Evaluate model using F1 and ROUGE metrics
    """
    bioasq_df = pd.read_csv("bioasq_combined.csv")
    np.random.seed(random_seed)
    eval_samples = bioasq_df.sample(n=num_samples)
    
    results = []
    
    for idx, row in eval_samples.iterrows():
        try:
            question = row['question']
            true_answer = row['text'].split('<answer>')[1].split('<context>')[0].strip()
            
            print(f"\nProcessing Question {len(results)+1}/{num_samples}:")
            print(f"Q: {question}")
            print(f"True Answer: {true_answer}")
            
            # Add delay between requests
            time.sleep(1)
            
            # Get model prediction
            try:
                result = process_medical_query_with_retry(train_medquad, question)
                predicted_answer = str(result['response'])
                print(f"Predicted Answer: {predicted_answer}")
            except Exception as e:
                print(f"Error processing question: {e}")
                predicted_answer = ""
                result = {'confidence': 'error'}
            
            # Calculate metrics
            metrics = evaluate_response(predicted_answer, true_answer)
            print(f"F1 Score: {metrics['f1_score']:.4f}")
            print(f"ROUGE-1: {metrics['rouge1']:.4f}")
            print(f"ROUGE-2: {metrics['rouge2']:.4f}")
            print(f"ROUGE-L: {metrics['rougeL']:.4f}")
            
            results.append({
                'question': question,
                'true_answer': true_answer,
                'predicted_answer': predicted_answer,
                **metrics,
                'confidence': result['confidence']
            })
            
            print("-" * 80)
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    return pd.DataFrame(results)

# Run evaluation
eval_results = evaluate_model(num_samples=50)

# Display evaluation results
print("\nEvaluation Summary:")
print("\nAverage Scores:")
print(f"F1 Score: {eval_results['f1_score'].mean():.4f}")
print(f"ROUGE-1: {eval_results['rouge1'].mean():.4f}")
print(f"ROUGE-2: {eval_results['rouge2'].mean():.4f}")
print(f"ROUGE-L: {eval_results['rougeL'].mean():.4f}")


Processing Question 1/50:
Q: Is the transcriptional regulator BACH1 an activator or a repressor?
True Answer: repressor
Predicted Answer: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "BACH1 (BTB and CNC homology 1) is primarily a transcriptional **repressor**.  It functions by binding to specific DNA sequences (MARE elements) and recruiting co-repressors, thereby inhibiting the transcription of target genes.  While some studies have hinted at context-dependent, limited activator roles, its predominant and well-established function is as a repressor.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.21149544838147286
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 6308,
        "candidates

In [59]:
def evaluate_model_test(num_samples=10, random_seed=42):
    """
    Evaluate model using symptom-related questions from test_medquad
    """
    # Filter symptom-related questions from test_medquad
    symptom_questions = test_medquad[test_medquad['Question'].str.contains('symptoms', case=False, na=False)]
    
    # Sample questions
    np.random.seed(random_seed)
    eval_samples = symptom_questions.sample(n=min(num_samples, len(symptom_questions)))
    
    results = []
    
    for idx, row in eval_samples.iterrows():
        try:
            question = row['Question']
            true_answer = row['Answer']
            
            print(f"\nProcessing Question {len(results)+1}/{num_samples}:")
            print(f"Q: {question}")
            print(f"True Answer: {true_answer}")
            
            # Add delay between requests
            time.sleep(1)
            
            # Get model prediction
            try:
                result = process_medical_query_with_retry(train_medquad, question)
                predicted_answer = str(result['response'])
                print(f"Predicted Answer: {predicted_answer}")
            except Exception as e:
                print(f"Error processing question: {e}")
                predicted_answer = ""
                result = {'confidence': 'error'}
            
            # Calculate metrics
            metrics = evaluate_response(predicted_answer, true_answer)
            print(f"F1 Score: {metrics['f1_score']:.4f}")
            print(f"ROUGE-1: {metrics['rouge1']:.4f}")
            print(f"ROUGE-2: {metrics['rouge2']:.4f}")
            print(f"ROUGE-L: {metrics['rougeL']:.4f}")
            
            results.append({
                'qa_id': row['qa_id'],
                'question': question,
                'true_answer': true_answer,
                'predicted_answer': predicted_answer,
                **metrics,
                'confidence': result['confidence']
            })
            
            print("-" * 80)
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    results_df = pd.DataFrame(results)
    
    # Print summary statistics
    print("\nEvaluation Summary:")
    print("\nAverage Scores:")
    print(f"F1 Score: {results_df['f1_score'].mean():.4f}")
    print(f"ROUGE-1: {results_df['rouge1'].mean():.4f}")
    print(f"ROUGE-2: {results_df['rouge2'].mean():.4f}")
    print(f"ROUGE-L: {results_df['rougeL'].mean():.4f}")
    
    return results_df

In [60]:
eval_results = evaluate_model_test(num_samples=10)


Processing Question 1/10:
Q: What are the symptoms of Glycogen storage disease type 13 ?
True Answer: What are the signs and symptoms of Glycogen storage disease type 13? Glycogen storage disease type 13 causes muscle pain (myalgia).  Individuals with GSD13 also experience exercise intolerance, which means they have difficulty exercising because they may have muscle weakness and tire easily. The Human Phenotype Ontology provides the following list of signs and symptoms for Glycogen storage disease type 13. If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Adult onset - Autosomal recessive inheritance - Elevated serum creatine phosphokinase - Exercise intolerance - Increased muscle glycogen content - Myalgia - The Human Phenotype Ontology (HPO) has collec

In [61]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt_tab')

def evaluate_response(predicted_answer, true_answer):
    """
    Evaluate response using BLEU and ROUGE metrics
    """
    results = {}
    
    # Calculate BLEU score
    reference = [word_tokenize(true_answer.lower())]
    candidate = word_tokenize(predicted_answer.lower())
    try:
        bleu_score = sentence_bleu(reference, candidate)
    except:
        bleu_score = 0
    results['bleu_score'] = bleu_score
    
    # Calculate ROUGE scores
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(true_answer, predicted_answer)
    
    results['rouge1'] = rouge_scores['rouge1'].fmeasure
    results['rouge2'] = rouge_scores['rouge2'].fmeasure
    results['rougeL'] = rouge_scores['rougeL'].fmeasure
    
    return results

def evaluate_model_test(num_samples=10, random_seed=42):
    """
    Evaluate model using symptom-related questions from test_medquad
    """
    # Filter symptom-related questions from test_medquad
    symptom_questions = test_medquad[test_medquad['Question'].str.contains('symptoms', case=False, na=False)]
    
    # Sample questions
    np.random.seed(random_seed)
    eval_samples = symptom_questions.sample(n=min(num_samples, len(symptom_questions)))
    
    results = []
    
    for idx, row in eval_samples.iterrows():
        try:
            question = row['Question']
            true_answer = row['Answer']
            
            print(f"\nProcessing Question {len(results)+1}/{num_samples}:")
            print(f"Q: {question}")
            print(f"True Answer: {true_answer}")
            
            # Add delay between requests
            time.sleep(1)
            
            # Get model prediction
            try:
                result = process_medical_query_with_retry(train_medquad, question)
                predicted_answer = str(result['response'])
                print(f"Predicted Answer: {predicted_answer}")
            except Exception as e:
                print(f"Error processing question: {e}")
                predicted_answer = ""
                result = {'confidence': 'error'}
            
            # Calculate metrics
            metrics = evaluate_response(predicted_answer, true_answer)
            print(f"BLEU Score: {metrics['bleu_score']:.4f}")
            print(f"ROUGE-1: {metrics['rouge1']:.4f}")
            print(f"ROUGE-2: {metrics['rouge2']:.4f}")
            print(f"ROUGE-L: {metrics['rougeL']:.4f}")
            
            results.append({
                'qa_id': row['qa_id'],
                'question': question,
                'true_answer': true_answer,
                'predicted_answer': predicted_answer,
                **metrics,
                'confidence': result['confidence']
            })
            
            print("-" * 80)
            
        except Exception as e:
            print(f"Error processing row {idx}: {e}")
            continue
    
    results_df = pd.DataFrame(results)
    
    # Print summary statistics
    print("\nEvaluation Summary:")
    print("\nAverage Scores:")
    print(f"BLEU Score: {results_df['bleu_score'].mean():.4f}")
    print(f"ROUGE-1: {results_df['rouge1'].mean():.4f}")
    print(f"ROUGE-2: {results_df['rouge2'].mean():.4f}")
    print(f"ROUGE-L: {results_df['rougeL'].mean():.4f}")
    
    return results_df

In [62]:
eval_results = evaluate_model_test(num_samples=10)


Processing Question 1/10:
Q: What are the symptoms of Glycogen storage disease type 13 ?
True Answer: What are the signs and symptoms of Glycogen storage disease type 13? Glycogen storage disease type 13 causes muscle pain (myalgia).  Individuals with GSD13 also experience exercise intolerance, which means they have difficulty exercising because they may have muscle weakness and tire easily. The Human Phenotype Ontology provides the following list of signs and symptoms for Glycogen storage disease type 13. If the information is available, the table below includes how often the symptom is seen in people with this condition. You can use the MedlinePlus Medical Dictionary to look up the definitions for these medical terms. Signs and Symptoms Approximate number of patients (when available) Adult onset - Autosomal recessive inheritance - Elevated serum creatine phosphokinase - Exercise intolerance - Increased muscle glycogen content - Myalgia - The Human Phenotype Ontology (HPO) has collec

d:\1UGM\SEM 5\COURSES\STBI\Tugas Akhir\healthcare-chatbot-using-qa-with-word-embedding-ir\.venv\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
d:\1UGM\SEM 5\COURSES\STBI\Tugas Akhir\healthcare-chatbot-using-qa-with-word-embedding-ir\.venv\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Predicted Answer: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The symptoms of deafness, dystonia, and cerebral hypomyelination are distinct and affect different bodily systems.  Let's examine each separately:\n\n\n**Deafness (Hearing Loss):**\n\nDeafness encompasses a range of hearing impairments, from mild difficulty hearing soft sounds to complete inability to hear.  Symptoms vary depending on the degree and type of hearing loss (conductive, sensorineural, or mixed).  Common symptoms include:\n\n* **Difficulty understanding speech:** Especially in noisy environments or when the speaker is far away.\n* **Need to turn up the volume:** On television, radio, or other audio devices.\n* **Tinnitus:** Ringing, buzzing, or other noises in the ears.\n* **Feeling of fullness or pressure in the ears:** This is

d:\1UGM\SEM 5\COURSES\STBI\Tugas Akhir\healthcare-chatbot-using-qa-with-word-embedding-ir\.venv\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Predicted Answer: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I apologize, but there is no information about Verloove Vanhorick Brubakk syndrome in the provided text or within my general medical knowledge base.  This appears to be a very rare or newly described condition that is not yet widely documented in medical literature.  To find information on this syndrome, I suggest searching medical databases like PubMed using the full name of the syndrome. You could also try searching for relevant publications on Google Scholar.  If you can provide additional information, such as a medical journal article or other source mentioning this syndrome, I may be able to provide more specific details.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
     

d:\1UGM\SEM 5\COURSES\STBI\Tugas Akhir\healthcare-chatbot-using-qa-with-word-embedding-ir\.venv\Lib\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


Predicted Answer: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The provided text does not contain information about Trichomegaly with intellectual disability, dwarfism, and pigmentary degeneration of the retina.  Therefore, I cannot answer your question using the provided context.  I would need additional medical literature or resources specific to this rare combination of conditions to provide a complete and accurate answer.\n\nTo find information about this, I recommend searching medical databases like PubMed using keywords such as \"Trichomegaly,\" \"intellectual disability,\" \"dwarfism,\" and \"pigmentary retinal degeneration.\"  You may also need to explore rare disease databases such as Orphanet.  It's crucial to consult with a medical professional for any health concerns and to receive accurate